In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import StandardScaler


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [4]:
from tensorflow import keras
from tensorflow.keras import layers

In [5]:
df = pd.read_csv('data/data-ready.csv')
df

,DateTime,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,z1,z2,z3,Year,...,Day,Hour,IsWeekend,IsNight,HourCos,HourSin,MonthCos,MonthSin,DayCos,DaySin
0,2017-01-01 00:00:00,6.559,73.8,0.083,0.051,0.119,34055.69620,16128.87538,20240.96386,2017,...,1,0,1,1,1.000000,0.000000,0.866025,5.000000e-01,0.97953,0.201299
1,2017-01-01 00:10:00,6.414,74.5,0.083,0.070,0.085,29814.68354,19375.07599,20131.08434,2017,...,1,0,1,1,1.000000,0.000000,0.866025,5.000000e-01,0.97953,0.201299
2,2017-01-01 00:20:00,6.313,74.5,0.080,0.062,0.100,29128.10127,19006.68693,19668.43373,2017,...,1,0,1,1,1.000000,0.000000,0.866025,5.000000e-01,0.97953,0.201299
3,2017-01-01 00:30:00,6.121,75.0,0.083,0.091,0.096,28228.86076,18361.09422,18899.27711,2017,...,1,0,1,1,1.000000,0.000000,0.866025,5.000000e-01,0.97953,0.201299
4,2017-01-01 00:40:00,5.921,75.7,0.081,0.048,0.085,27335.69620,17872.34043,18442.40964,2017,...,1,0,1,1,1.000000,0.000000,0.866025,5.000000e-01,0.97953,0.201299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52411,2017-12-30 23:10:00,7.010,72.4,0.080,0.040,0.096,31160.45627,26857.31820,14780.31212,2017,...,30,23,1,1,0.965926,-0.258819,1.000000,-2.449294e-16,0.97953,-0.201299
52412,2017-12-30 23:20:00,6.947,72.6,0.082,0.051,0.093,30430.41825,26124.57809,14428.81152,2017,...,30,23,1,1,0.965926,-0.258819,1.000000,-2.449294e-16,0.97953,-0.201299
52413,2017-12-30 23:30:00,6.900,72.8,0.086,0.084,0.074,29590.87452,25277.69254,13806.48259,2017,...,30,23,1,1,0.965926,-0.258819,1.000000,-2.449294e-16,0.97953,-0.201299
52414,2017-12-30 23:40:00,6.758,73.0,0.080,0.066,0.089,28958.17490,24692.23688,13512.60504,2017,...,30,23,1,1,0.965926,-0.258819,1.000000,-2.449294e-16,0.97953,-0.201299


In [6]:
df = df.drop("DateTime", axis=1)

In [7]:
df_z1 = df.drop(["z2", "z3"], axis=1)
df_z2 = df.drop(["z1", "z3"], axis=1)
df_z3 = df.drop(["z1", "z2"], axis=1)

In [21]:
zones = [df_z1, df_z2, df_z3]
targets = ["z1", "z2", "z3"]

In [22]:
R_2s = []
MSEs = []

In [23]:
for zone, target in zip(zones, targets):
    print(f"===================  {target}  ==================")
    X = zone.drop(target, axis=1)
    y = zone[target]

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scale the features using StandardScaler
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    target_scaler = StandardScaler()
    y_train = target_scaler.fit_transform(y_train.values.reshape(-1, 1)).flatten()
    y_test = target_scaler.transform(y_test.values.reshape(-1, 1)).flatten()

    # Define the ANN model with dropout and early stopping
    model = keras.Sequential([
        layers.Dense(32, activation="relu", input_shape=(X_train.shape[1],)),  # Input layer
        layers.Dropout(0.2),  # Dropout layer with 20% dropout rate
        layers.Dense(16, activation="tanh"),  # Hidden layer
        layers.Dropout(0.2),
        layers.Dense(1)  # Output layer for regression
    ])

    # Compile the model (specifies loss function and optimizer)
    model.compile(loss="mae", optimizer="adam")  # Use 'mse' for regression

    # Train the model
    model.fit(X_train, y_train, epochs=100, batch_size=16)  # Adjust epochs and batch size

    # Use the model for prediction
    predictions = model.predict(X_test)

    
    # Calculate evaluation metrics
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    
    R_2s.append(r2)
    MSEs.append(mse)
    


===================  z1  ==================
Epoch 1/100


/Users/abdelhak/anaconda3/envs/AI_ENV/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 255us/step - loss: 0.5133
Epoch 2/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 239us/step - loss: 0.3211
Epoch 3/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 235us/step - loss: 0.2912
Epoch 4/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 237us/step - loss: 0.2834
Epoch 5/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 235us/step - loss: 0.2757
Epoch 6/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 243us/step - loss: 0.2737
Epoch 7/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 237us/step - loss: 0.2711
Epoch 8/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 236us/step - loss: 0.2673
Epoch 9/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 234us/step - loss: 0.2649
Epoch 10/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 262us/step - loss: 0.2622
Epoch 11/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 248us/step - loss: 0.2633
Epoch 12/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 245us/step - loss: 0.2621
Epoch 13/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 259us/step - loss: 0.2615
Epoch 14/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 240us/step - loss: 0.2609

/Users/abdelhak/anaconda3/envs/AI_ENV/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 237us/step - loss: 0.5285
Epoch 2/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 234us/step - loss: 0.3682
Epoch 3/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 234us/step - loss: 0.3370
Epoch 4/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 234us/step - loss: 0.3204
Epoch 5/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 244us/step - loss: 0.3113
Epoch 6/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 235us/step - loss: 0.3023
Epoch 7/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 234us/step - loss: 0.2969
Epoch 8/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 234us/step - loss: 0.2939
Epoch 9/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 234us/step - loss: 0.2915
Epoch 10/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 243us/step - loss: 0.2880
Epoch 11/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 234us/step - loss: 0.2852
Epoch 12/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 236us/step - loss: 0.2845
Epoch 13/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 234us/step - loss: 0.2819
Epoch 14/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 233us/step - loss: 0.2798

/Users/abdelhak/anaconda3/envs/AI_ENV/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 244us/step - loss: 0.4649
Epoch 2/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 238us/step - loss: 0.3009
Epoch 3/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 248us/step - loss: 0.2642
Epoch 4/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 240us/step - loss: 0.2478
Epoch 5/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 238us/step - loss: 0.2347
Epoch 6/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 239us/step - loss: 0.2281
Epoch 7/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 238us/step - loss: 0.2287
Epoch 8/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 247us/step - loss: 0.2208
Epoch 9/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 239us/step - loss: 0.2209
Epoch 10/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 239us/step - loss: 0.2191
Epoch 11/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 238us/step - loss: 0.2155
Epoch 12/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 238us/step - loss: 0.2129
Epoch 13/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 248us/step - loss: 0.2148
Epoch 14/100
2621/2621 ━━━━━━━━━━━━━━━━━━━━ 1s 237us/step - loss: 0.2127

In [24]:
i=1
for r2, mse in zip(R_2s, MSEs):
    # Print evaluation metrics
    print(f"===================  z{i}  ==================")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"R-squared (R^2): {r2:.4f}")
    i+=1

===================  z1  ==================
Mean Squared Error (MSE): 0.0652
R-squared (R^2): 0.9342
===================  z2  ==================
Mean Squared Error (MSE): 0.0719
R-squared (R^2): 0.9279
===================  z3  ==================
Mean Squared Error (MSE): 0.0350
R-squared (R^2): 0.9648
